In [12]:
import pandas as pd

# データフレームの作成
data = {
    'Points limit': [500, 1000, 10000, 500, 1000, 10000],
    'Proposed (5%)': [1, 9, 0.083, 25, 33, 0.106],
    'ORB': [2, 10, 0.361, 26, 34, 0.576],
    # 他の列も同様に追加
}
df = pd.DataFrame(data)

# LaTeX形式で基本的なテーブルを出力
latex_table = df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrr}
\toprule
 Points limit &  Proposed (5\%) &    ORB \\
\midrule
          500 &          1.000 &  2.000 \\
         1000 &          9.000 & 10.000 \\
        10000 &          0.083 &  0.361 \\
          500 &         25.000 & 26.000 \\
         1000 &         33.000 & 34.000 \\
        10000 &          0.106 &  0.576 \\
\bottomrule
\end{tabular}



C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2096336193.py:13: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False)


In [33]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import math

COMPARE_DESCRIPTORS = ["Proposed", "orb", "sphorb", "sift", "spoint", "alike", ]
ABLATION_DESCRIPTORS = ["Proposed","Proposed01", "Proposed1", "Proposed10", "Proposed_un", "Ltspoint", "tspoint", "Ftspoint"] 
NOT_T_DESCRIPTORS = ["spoint", "tspoint", "sphorb", "Ltspoint", "Proposed01", "Proposed03", "Proposed05", "Proposed1", "Proposed3", "Proposed", "Proposed10", "Proposed20", "Proposed_un", "Ftspoint"]
METHODS = ["", "t"]
PARAMS = ["R", "T"]
PARAMS_DICT = {"R": "Rotation", "T": "Translation"}
ALL_LOCS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2", "Urban1", "Urban2", "Urban3", "Urban4"]
INDOORS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2"]
OUTDOORS = ["Urban1", "Urban2", "Urban3", "Urban4"]
TIME_CATEGORYS = ["FP", "MC", "PE"]
DESCRIPTORS_DICT = {"Proposed1": "Proposed (1%)",
                    "Proposed01": "Proposed (0.1%)",
                    "Proposed03": "Proposed (0.3%)",
                    "Proposed05": "Proposed (0.5%)",
                    "Proposed3": "Proposed (3%)",
                    "Proposed": "Proposed (5%)",
                    "Proposed10": "Proposed (10%)",
                    "Proposed20": "Proposed (20%)",
                    "Proposed_un": "MNN",
                    "orb": "ORB", 
                    "sphorb": "SPHORB",
                    "sift": "SIFT",
                    "alike": "ALIKE",
                    "spoint": "SPoint",
                    "tspoint": "KNN+L",
                    "Ltspoint": "Proposed (5%)+L",
                    "Ftspoint": "FLANN"
                    }
METHODS_DICT = {"t": "T", "":""}

In [34]:
def read_csv_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        data = [float(row[0]) for row in reader]
    return data

In [35]:
def make_latex_table_MAEMSE():
    for i, param in enumerate(PARAMS):
        for j, loc in enumerate(["indoor", "outdoor"]):
            if loc == "indoor":
                locs = INDOORS
            else:
                locs = OUTDOORS
            print(param, loc)
            maemse_data = {'Points limit': [500, 1000, 10000, 500, 1000, 10000]}
            maemse_compare_df = pd.DataFrame(maemse_data)
            maemse_ablation_df = pd.DataFrame(maemse_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    maemse_df = maemse_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        maemse_df[method_tmp] = [0, 0, 0, 0, 0, 0]
        
            for idx_points, num_points in enumerate([500, 1000, 10000]):
                base_path = f"results/FP_{num_points}/values"
                value_dict = {}
                for tgt in ["compare", "ablation"]:
                    if tgt == "compare":
                        descriptors = COMPARE_DESCRIPTORS
                        maemse_df = maemse_compare_df
                    else:
                        descriptors = ABLATION_DESCRIPTORS
                        maemse_df = maemse_ablation_df
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            all_error_data = []
                            for scene in locs:
                                file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/{param}_ERRORS.csv"
                                error_data = read_csv_data(file_path)
                                all_error_data.extend(error_data)
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            maemse_df.loc[idx_points, method_tmp] = np.mean(all_error_data)
                            maemse_df.loc[idx_points+3, method_tmp] = np.mean(np.array(all_error_data) ** 2)
        
            maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(maemse_latex_table)
            maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(maemse_latex_table)

make_latex_table_MAEMSE()

R indoor
\begin{tabular}{rrrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
          500 &          0.083 & 0.914 & 0.667 &   0.786 & 0.544 &  0.411 &   0.758 &  2.975 &   2.782 \\
         1000 &          0.092 & 0.832 & 0.559 &   0.526 & 0.281 &  0.229 &   0.734 &  2.923 &   2.681 \\
        10000 &          0.083 & 0.361 & 0.184 &   0.246 & 0.129 &  0.125 &   0.758 &  2.847 &   2.677 \\
          500 &          0.078 & 1.860 & 1.218 &   1.685 & 0.932 &  0.868 &   2.176 &  9.305 &   8.650 \\
         1000 &          0.123 & 1.811 & 1.144 &   1.082 & 0.378 &  0.495 &   2.057 &  9.129 &   8.300 \\
        10000 &          0.106 & 0.576 & 0.276 &   0.460 & 0.105 &  0.214 &   2.099 &  8.872 &   8.286 \\
\bottomrule
\end{tabular}



C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\3809608482.py:49: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\3809608482.py:51: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\3809608482.py:49

\begin{tabular}{rrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
          500 &          0.083 &            0.107 &          0.088 &           1.091 & 1.091 &            1.804 &  1.091 &  0.769 \\
         1000 &          0.092 &            0.075 &          0.086 &           1.060 & 1.060 &            1.994 &  1.060 &  0.616 \\
        10000 &          0.083 &            0.071 &          0.085 &           1.059 & 1.059 &            1.994 &  1.059 &  0.634 \\
          500 &          0.078 &            0.111 &          0.090 &           3.232 & 3.232 &            5.600 &  3.232 &  2.215 \\
         1000 &          0.123 &            0.048 &          0.106 &           3.101 & 3.101 &            6.182 &  3.101 &  1.633 \\
        10000 &          0.106 &            0.046 &          0.105 &           3.091 & 3.091 &            6.182 &  3.091 &  1.672 \\
\bottomrule
\end{ta

C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\3809608482.py:49: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\3809608482.py:51: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\3809608482.py:49

In [36]:
def make_latex_table_time():
    for i, param in enumerate(PARAMS):
        for j, loc in enumerate(["indoor", "outdoor"]):
            if loc == "indoor":
                locs = INDOORS
            else:
                locs = OUTDOORS
            time_data = {'Points limit': ["ALL", 500, 1000, 10000]}
            time_compare_df = pd.DataFrame(time_data)
            time_ablation_df = pd.DataFrame(time_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    time_df = time_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    time_df = time_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        time_df[method_tmp] = [0, 0, 0, 0]
        
            for idx_points, num_points in enumerate([500, 1000, 10000]):
                base_path = f"results/FP_{num_points}/values"
                value_dict = {}
                for tgt in ["compare", "ablation"]:
                    if tgt == "compare":
                        descriptors = COMPARE_DESCRIPTORS
                        time_df = time_compare_df
                    else:
                        descriptors = ABLATION_DESCRIPTORS
                        time_df = time_ablation_df
                    for time_category in TIME_CATEGORYS:
                        #print(time_category)
                        for descriptor in descriptors:
                            for method in METHODS:
                                if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                    continue
                                method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                                all_time_data = []
                                for scene in ALL_LOCS:
                                    file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/TIMES_{time_category}.csv"
                                    time_data = read_csv_data(file_path)
                                    all_time_data.extend(time_data)
                                if time_category == "FP":
                                    time_df.loc[0, method_tmp] = np.mean(all_time_data)
                                elif time_category == "MC":
                                    time_df.loc[idx_points+1, method_tmp] = np.mean(all_time_data)
        
            time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(time_latex_table)
            time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(time_latex_table)

make_latex_table_time()

C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)


\begin{tabular}{lrrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.526 & 0.231 & 0.226 &   0.287 & 0.230 &  0.684 &   0.262 &  0.191 &   0.337 \\
         500 &          0.010 & 0.011 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.016 & 0.010 & 0.013 &   0.002 & 0.010 &  0.010 &   0.010 &  0.010 &   0.010 \\
       10000 &          0.020 & 0.084 & 0.057 &   0.104 & 0.012 &  0.011 &   0.014 &  0.012 &   0.011 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
         ALL &          0.526 &            0.523 &          0.524 &           0.493 & 0.492 &            0.493 &  0.496 &  0.493 \\
         500 &          0.010 &            0.010 &          0.012 &           0.011 & 0

C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)


\begin{tabular}{lrrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.526 & 0.231 & 0.226 &   0.287 & 0.230 &  0.684 &   0.262 &  0.191 &   0.337 \\
         500 &          0.010 & 0.011 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.016 & 0.010 & 0.013 &   0.002 & 0.010 &  0.010 &   0.010 &  0.010 &   0.010 \\
       10000 &          0.020 & 0.084 & 0.057 &   0.104 & 0.012 &  0.011 &   0.014 &  0.012 &   0.011 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
         ALL &          0.526 &            0.523 &          0.524 &           0.493 & 0.492 &            0.493 &  0.496 &  0.493 \\
         500 &          0.010 &            0.010 &          0.012 &           0.011 & 0

C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)


\begin{tabular}{lrrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.526 & 0.231 & 0.226 &   0.287 & 0.230 &  0.684 &   0.262 &  0.191 &   0.337 \\
         500 &          0.010 & 0.011 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.016 & 0.010 & 0.013 &   0.002 & 0.010 &  0.010 &   0.010 &  0.010 &   0.010 \\
       10000 &          0.020 & 0.084 & 0.057 &   0.104 & 0.012 &  0.011 &   0.014 &  0.012 &   0.011 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
         ALL &          0.526 &            0.523 &          0.524 &           0.493 & 0.492 &            0.493 &  0.496 &  0.493 \\
         500 &          0.010 &            0.010 &          0.012 &           0.011 & 0

C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:52: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_22648\2099619734.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
